In [ ]:
import pandas as pd
import numpy as np
import tqdm
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))


from pprint import pprint


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel, CoherenceModel


In [2]:
df = pd.read_csv('feed.csv')

In [16]:
projects = []

for i in range(len(df)):
    project = str(df['project_description'][i]) + str(df['fields_of_science'][i]) + str(df['keywords'][i])
    projects.append(project)

In [17]:
from helpers import preprocess_text
preprocessed_projects = [preprocess_text(project) for project in tqdm.tqdm(projects)]

  0%|          | 0/503 [00:00<?, ?it/s]

100%|██████████| 503/503 [00:00<00:00, 1366.19it/s]


In [19]:
# convert to series for easier manipulation
preprocessed_projects = pd.Series(preprocessed_projects)

# create Dictionary from preprocessed articles
dictionary = Dictionary(preprocessed_projects)

# filter out tokens that appear too much or too little
dictionary.filter_extremes(no_below=2, no_above=0.5, keep_n=150000)


In [34]:
projects_bow = [dictionary.doc2bow(doc) for doc in preprocessed_projects]

In [35]:
# create TF-IDF encoder from BoW corpus
tfidf_encoder = TfidfModel(projects_bow)

# weighted tfidf corpus
projects_tfidf = tfidf_encoder[projects_bow]


In [28]:
# Apply LDA with 12 topics
lda_model = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=10,
                     passes=10,
                     iterations=100)


In [29]:
topics = lda_model.show_topics(num_topics=10, num_words=8)
for topic in topics:
    print(topic)

(0, '0.039*"bird" + 0.038*"survey" + 0.030*"population" + 0.020*"help" + 0.019*"wildlife" + 0.018*"state" + 0.017*"scientist" + 0.017*"animal"')
(1, '0.038*"data" + 0.028*"climate" + 0.016*"weather" + 0.014*"change" + 0.014*"time" + 0.013*"project" + 0.013*"earth" + 0.012*"system"')
(2, '0.036*"data" + 0.021*"weather" + 0.020*"sea" + 0.019*"project" + 0.018*"change" + 0.017*"climate" + 0.016*"scientist" + 0.015*"animal"')
(3, '0.044*"park" + 0.030*"specie" + 0.030*"national" + 0.021*"ecology" + 0.019*"environment" + 0.019*"pollinator" + 0.017*"scientist" + 0.016*"plant"')
(4, '0.044*"community" + 0.033*"project" + 0.028*"research" + 0.024*"system" + 0.017*"environmental" + 0.014*"developing" + 0.014*"use" + 0.013*"study"')
(5, '0.051*"air" + 0.043*"sensor" + 0.030*"data" + 0.025*"quality" + 0.024*"community" + 0.020*"cost" + 0.019*"epa" + 0.018*"low"')
(6, '0.064*"park" + 0.063*"education" + 0.057*"national" + 0.054*"bioblitz" + 0.042*"specie" + 0.020*"group" + 0.020*"bird" + 0.018*"ex

In [33]:
lda_model.get_document_topics(corpus_bow[50])

[(5, 0.55972165), (9, 0.42598894)]

In [31]:
pprint(projects[200])

('The Youth Forest Monitoring Program (YFMP) is an intensive 7-week summer '
 'internship for high school students. Participants explore forest ecology, '
 'investigate natural resource careers, improve forest health through '
 'stewardship projects, and supplement forest analysis through citizen-led '
 'forest health monitoring.Ecology and environment,  Nature and outdoorsforest '
 'ecology, forest health, natural resource careers')
